In [2]:
import cv2
import imutils
import numpy as np
import os

# Global variables
bg = None

def run_avg(image, aWeight):
    global bg
    # Initialize the background
    if bg is None:
        bg = image.copy().astype("float")
        return

    # Compute weighted average, accumulate it, and update the background
    cv2.accumulateWeighted(image, bg, aWeight)

def segment(image, threshold=25):
    global bg
    # Find the absolute difference between background and current frame
    diff = cv2.absdiff(bg.astype("uint8"), image)

    # Threshold the diff image so that we get the foreground
    thresholded = cv2.threshold(diff,
                                threshold,
                                255,
                                cv2.THRESH_BINARY)[1]

    # Get the contours in the thresholded image
    (cnts, _) = cv2.findContours(thresholded.copy(),
                                 cv2.RETR_EXTERNAL,
                                 cv2.CHAIN_APPROX_SIMPLE)

    # Return None if no contours detected
    if len(cnts) == 0:
        return
    else:
        # Based on contour area, get the maximum contour which is the hand
        segmented = max(cnts, key=cv2.contourArea)
        return (thresholded, segmented)

gestures = ["Blank", "Fist", "Ok", "Palm", "ThumbsUp", "ThumbsDown"]
dataset_dir = "Dataset/"
# Get the reference to the webcam
camera = cv2.VideoCapture(0)

def main():
    # Initialize weight for running average
    aWeight = 0.5

    # Region of interest (ROI) coordinates
    top, right, bottom, left = 10, 350, 225, 590

    # Initialize num of frames
    num_frames = 0
    image_num = 1

    # Loop through different camera angles
    for angle in range(0, 360, 15):  # Adjust the step size for different angles
        # Set the camera angle
        camera.set(0, angle)

        # Keep looping until interrupted
        while True:
            # Get the current frame
            (grabbed, frame) = camera.read()
            if grabbed:
                # Loop through different orientations
                for orientation in range(-30, 31, 15):  # Adjust the step size for different orientations
                    # Set the orientation
                    rotated_frame = imutils.rotate(frame, orientation)

                    # Resize the frame
                    rotated_frame = imutils.resize(rotated_frame, width=700)

                    # Flip the frame so that it is not the mirror view
                    rotated_frame = cv2.flip(rotated_frame, 1)

                    # Clone the frame
                    clone = rotated_frame.copy()

                    # Get the height and width of the frame
                    (height, width) = rotated_frame.shape[:2]

                    # Get the ROI
                    roi = rotated_frame[top:bottom, right:left]

                    # Convert the ROI to grayscale and blur it
                    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                    gray = cv2.GaussianBlur(gray, (7, 7), 0)

                    # To get the background, keep looking till a threshold is reached
                    # so that our running average model gets calibrated
                    if num_frames < 30:
                        run_avg(gray, aWeight)
                    else:
                        # Segment the hand region
                        hand = segment(gray)

                        # Check whether hand region is segmented
                        if hand is not None:
                            # If yes, unpack the thresholded image and
                            # segmented region
                            (thresholded, segmented) = hand

                            # Draw the segmented region and display the frame
                            cv2.drawContours(
                                clone, [segmented + (right, top)], -1, (0, 0, 255))

                            # Mention the directory in which you wanna store the images followed by the image name
                            output_dir = os.path.join(dataset_dir, f"{gestures[0]}Test/")
                            output_path = os.path.join(output_dir, f"{gestures[0]}_{image_num}.png")
                            cv2.imwrite(output_path, thresholded)
                            image_num += 1

                    # Draw the segmented hand
                    cv2.rectangle(clone, (left, top), (right, bottom), (0, 255, 0), 2)

                    # Increment the number of frames
                    num_frames += 1

                    # Display the frame with segmented hand
                    cv2.imshow("Video Feed", clone)

                    # Observe the keypress by the user
                    keypress = cv2.waitKey(1) & 0xFF

                    # Start recording when 's' key is pressed
                    if keypress == ord("s"):
                        break

            else:
                print("[Warning!] Error input, Please check your (camera or video)")
                break

    # Free up memory
    camera.release()
    cv2.destroyAllWindows()

# Call the main function
main()


KeyboardInterrupt: 

: 